In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras import layers
from keras.layers import Conv2D
from keras.layers import Input
from keras.layers import MaxPooling2D, UpSampling2D
from keras.models import Model
from numpy import reshape
import matplotlib.pyplot as plt

In [2]:
dir = "images_train"
xtrain = tf.keras.utils.image_dataset_from_directory(
    directory = dir,
    labels="inferred",
    label_mode="int",
    class_names=['A','B','C','D','E','F','G','H','I','J','K','L'],
    color_mode="rgb",
    batch_size=32,
    image_size=(32, 32),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    
)

Found 176066 files belonging to 12 classes.


In [3]:
def resize_image(x):
    return tf.image.resize(x, (32, 32))

# Redimensionnement des images après les avoir extraites du dataset
auto_train = xtrain.map(lambda x,y: (x, x))
auto_train = auto_train.map(lambda x, y: (resize_image(x), resize_image(x)))



input_img = Input(shape=(32, 32, 3))

enc_conv1 = Conv2D(12, (3, 3), activation='relu', padding='same')(input_img)
enc_pool1 = MaxPooling2D((2, 2), padding='same')(enc_conv1)
enc_conv2 = Conv2D(10, (4, 4), activation='relu', padding='same')(enc_pool1)
enc_pool2 = MaxPooling2D((4, 4), padding='same')(enc_conv2)
enc_conv3 = Conv2D(8, (4, 4), activation='relu', padding='same')(enc_pool2)
enc_ouput = MaxPooling2D((4, 4), padding='same')(enc_conv2)

dec_conv2 = Conv2D(8, (4, 4), activation='relu', padding='same')(enc_ouput)
dec_upsample2 = UpSampling2D((4, 4))(dec_conv2)
dec_conv3 = Conv2D(12, (3, 3), activation='relu', padding='same')(dec_upsample2)
dec_upsample3 = UpSampling2D((2, 2))(dec_conv3)
dec_conv4 = Conv2D(10, (3, 3), activation='relu', padding='same')(dec_upsample3)
dec_upsample4 = UpSampling2D((2, 2))(dec_conv3)
dec_output = Conv2D(3, (3, 3), activation='relu', padding='same')(dec_upsample4)

In [4]:
autoencoder = Model(input_img, dec_output)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 12)        336       
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 12)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 10)        1930      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 4, 4, 10)          0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 4, 4, 8)           1288  

In [5]:
autoencoder.fit(auto_train, epochs=3, batch_size=128, shuffle=True)

Epoch 1/3
5503/5503 [==============================] - 380s 69ms/step - loss: 296.3689
Epoch 2/3
5503/5503 [==============================] - 205s 37ms/step - loss: 166.3442
Epoch 3/3
5503/5503 [==============================] - 206s 37ms/step - loss: 156.0096


In [6]:
autoencoder.save('conv_autoencoder_model_v4.h5')

C:\Users\nicop\anaconda3\envs\tensorflow\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
